In [78]:
import pandas as pd
import ast
import numpy as np
import os

In [123]:
eval_results_path = "/Users/Maria.Tigina/PycharmProjects/lca-baselines/bug_localization/data/eval"

In [181]:
run_id = "togetherai_filepath"

In [182]:
df = pd.read_json(os.path.join(eval_results_path, run_id, 'results.jsonl'), lines=True)
df

,text_id,All Files Count,Expected Bug Files Count,Actual Bug Files Count,% of Bug Files,Precision,Recall,F1 Score,FPR,All Correct,At Least One Correct,All Incorrect,system_prompt_tokens,user_prompt_tokens,total_tokens
0,thealgorithms/python/295/289,195,1,1,0.005128,1.000000,1.0,1.000000,0.000000,1,1,0,116,1921,2039
1,keras-team/keras/15943/15942,553,1,2,0.003617,0.500000,1.0,0.666667,0.001812,0,1,0,116,6370,6488
2,serverless/serverless/5500/5499,10,1,1,0.100000,1.000000,1.0,1.000000,0.000000,1,1,0,116,265,383
3,textualize/rich/2212/2197,122,1,1,0.008197,1.000000,1.0,1.000000,0.000000,1,1,0,116,1659,1777
4,mitmproxy/mitmproxy/4298/3994,307,2,2,0.006515,0.500000,0.5,0.500000,0.003279,0,1,0,116,3191,3309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,zalando/nakadi/677/391,299,1,4,0.013378,0.250000,1.0,0.400000,0.010067,0,1,0,116,5551,5669
95,apicurio/apicurio-studio/1604/1533,306,1,3,0.009804,0.333333,1.0,0.500000,0.006557,0,1,0,116,7961,8079
96,objectionary/eo/1263/1256,156,1,1,0.006410,1.000000,1.0,1.000000,0.000000,1,1,0,116,2719,2837
97,spring-cloud/spring-cloud-consul/230/229,41,1,2,0.048780,0.500000,1.0,0.666667,0.025000,0,1,0,116,1355,1473


In [183]:
df[['Precision', 'Recall', 'F1 Score', 'FPR']].describe().round(3)

,Precision,Recall,F1 Score,FPR
count,99.000,99.000,99.000,99.000
mean,0.569,0.652,0.548,0.022
std,0.331,0.355,0.290,0.102
min,0.000,0.000,0.000,0.000
25%,0.333,0.333,0.400,0.000
50%,0.500,0.667,0.500,0.002
75%,1.000,1.000,0.667,0.010
max,1.000,1.000,1.000,1.000


In [184]:
df[['All Correct', 'At Least One Correct', 'All Incorrect', 'Actual Bug Files Count']].describe().round(2)

,All Correct,At Least One Correct,All Incorrect,Actual Bug Files Count
count,99.00,99.00,99.00,99.00
mean,0.19,0.92,0.08,2.88
std,0.40,0.27,0.27,1.76
min,0.00,0.00,0.00,1.00
25%,0.00,1.00,0.00,1.50
50%,0.00,1.00,0.00,3.00
75%,0.00,1.00,0.00,4.00
max,1.00,1.00,1.00,12.00


In [185]:
df[['All Files Count', 'Expected Bug Files Count', 'Actual Bug Files Count', '% of Bug Files']].describe().round(3)

,All Files Count,Expected Bug Files Count,Actual Bug Files Count,% of Bug Files
count,99.000,99.000,99.000,99.000
mean,649.222,2.727,2.879,0.054
std,1178.726,2.702,1.763,0.173
min,1.000,1.000,1.000,0.000
25%,79.500,1.000,1.500,0.003
50%,242.000,2.000,3.000,0.010
75%,532.000,3.000,4.000,0.029
max,6473.000,14.000,12.000,1.000


In [159]:
df[['total_tokens']].describe().round(3)

,total_tokens
count,150.000
mean,11150.080
std,19303.041
min,251.000
25%,1908.750
50%,4031.500
75%,9831.000
max,100001.000


In [160]:
# max 3324 min 149, mean 611

In [ ]:
# max >200000 min 251, mean 11150